In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%env KAGGLE_USERNAME=mckharris
%env KAGGLE_KEY=XXXX

!kaggle datasets download -d samratp/bikeshare-analysis
!unzip -q bikeshare-analysis.zip

In [3]:
Washington = pd.read_csv("Washington-CapitalBikeshare-2016.csv")
NYC = pd.read_csv("NYC-CitiBike-2016.csv")
Chicago = pd.read_csv("Chicago-Divvy-2016.csv")

In [4]:
NYC["usertype"] = NYC["usertype"].replace("Customer", "Casual")
NYC = pd.get_dummies(NYC, columns=['usertype'], prefix='usertype', drop_first=True)
NYC = pd.get_dummies(NYC, columns=['gender'], prefix='gender', drop_first=True)
NYC['is_customer'] = (NYC['usertype_Subscriber'] == 1).astype(int)
NYC['gender_1'] = NYC['gender_1'].astype(int)
NYC = NYC.drop(columns = ['usertype_Subscriber', 'gender_2'])
NYC.rename(columns={'is_customer': 'user_type'}, inplace=True)

In [5]:
NYC['starttime'] = pd.to_datetime(NYC['starttime'])
NYC['stoptime'] = pd.to_datetime(NYC['stoptime'])
NYC['start_hour'] = NYC['starttime'].dt.hour
NYC['start_minute'] = NYC['starttime'].dt.minute
NYC['start_day'] = NYC['starttime'].dt.day
NYC['start_month'] = NYC['starttime'].dt.month
NYC['start_year'] = NYC['starttime'].dt.year

NYC['stop_hour'] = NYC['stoptime'].dt.hour
NYC['stop_minute'] = NYC['stoptime'].dt.minute
NYC['stop_day'] = NYC['stoptime'].dt.day
NYC['stop_month'] = NYC['stoptime'].dt.month
NYC['stop_year'] = NYC['stoptime'].dt.year

NYC = NYC.drop(columns=['starttime'])
NYC = NYC.drop(columns=['stoptime'])
NYC = NYC.drop(columns=['start station name'])
NYC = NYC.drop(columns=['end station name'])

NYC = NYC.dropna()

In [6]:
cols = ['bikeid'] + [col for col in NYC.columns if col != 'bikeid']
NYC = NYC[cols]

cols = ['user_type'] + [col for col in NYC.columns if col != 'user_type']
NYC = NYC[cols]

In [7]:
Q1 = NYC['tripduration'].quantile(0.25)
Q3 = NYC['tripduration'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

NYC = NYC[(NYC['tripduration'] >= lower_bound) & (NYC['tripduration'] <= upper_bound)]

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint

In [9]:
X  = NYC.drop(columns = ["tripduration", "start_hour", "start_minute", "stop_hour", "stop_minute"])
y = NYC["tripduration"]

In [10]:
y_log = np.log1p(y)

In [11]:
X_train, X_test, y_train_log, y_test_log = train_test_split(
    X, y_log, test_size=0.2, random_state=42)

In [12]:
pip install xgboost

In [13]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold

In [14]:
XGB_model = XGBRegressor(
    max_depth=30,
    min_child_weight=2,
    gamma=5,
    n_estimators=200,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

In [15]:
XGB_model.fit(X_train, y_train_log)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=30, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [16]:
y_pred_log = XGB_model.predict(X_test)

In [17]:
mse = mean_squared_error(y_test_log, y_pred_log)
r2 = r2_score(y_test_log, y_pred_log)

print("MSE: ", mse)
print("R^2 Score: ", r2)

MSE:  0.1478395710966301
R^2 Score:  0.6342718275203019


In [15]:
from sklearn.model_selection import GridSearchCV

In [18]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10, 15],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

In [20]:
XGB_model_tuned = XGBRegressor(tree_method='gpu_hist', use_label_encoder=False, random_state = 42)

In [25]:
grid_search = GridSearchCV(estimator = XGB_model_tuned, param_grid = param_grid, cv = 3, scoring = 'neg_mean_squared_error', verbose = 1, n_jobs = 4)

In [27]:
grid_search.fit(X_train, y_train_log)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


KeyboardInterrupt: 

In [28]:
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

Best hyperparameters: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 15, 'n_estimators': 200, 'subsample': 0.8}


In [29]:
best_model = grid_search.best_estimator_

y_pred_log = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test_log, y_pred_log)
r2 = r2_score(y_test_log, y_pred_log)

print("MSE: ", mse)
print("R² Score: ", r2)

MSE:  0.11262896393078611
R² Score:  0.7213764566472881


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:08:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:08:05] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [16]:
XGB_model_tuned2 = XGBRegressor(
    tree_method='hist',
    device='cuda',
    use_label_encoder=False,
    random_state=42
)

In [19]:
grid_search2 = GridSearchCV(estimator = XGB_model_tuned2, param_grid = param_grid, cv = 3, scoring = 'neg_mean_squared_error', verbose = 1, n_jobs = 4)

In [21]:
grid_search2.fit(X_train, y_train_log)

Fitting 3 folds for each of 540 candidates, totalling 1620 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:45:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device='cuda',
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=4,
             param_grid={'colsample_bytree': [0.8, 0.9, 1],
                         'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': [3, 5, 7, 10, 15],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 0.9, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [22]:
best_params2 = grid_search2.best_params_
print("Best hyperparameters:", best_params2)

Best hyperparameters: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 15, 'n_estimators': 200, 'subsample': 0.8}


In [24]:
best_model = grid_search2.best_estimator_

y_pred_log = best_model.predict(X_test)

mse = mean_squared_error(y_test_log, y_pred_log)
r2 = r2_score(y_test_log, y_pred_log)

print("MSE: ", mse)
print("R² Score: ", r2)

MSE:  0.11262896393078611
R² Score:  0.7213764566472881


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:23:49] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [14]:
XGB_model_testing_depth = XGBRegressor(max_depth = 20, n_estimators = 200, learning_rate = 0.05, subsample = 0.8, colsample_bytree = 1, random_state = 42)

In [33]:
XGB_model_testing_depth.fit(X_train, y_train_log)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=20,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [34]:
y_pred_log = XGB_model_testing_depth.predict(X_test)

In [35]:
mse = mean_squared_error(y_test_log, y_pred_log)
r2 = r2_score(y_test_log, y_pred_log)

print("MSE: ", mse)
print("R² Score: ", r2)

MSE:  0.11612869942805458
R² Score:  0.7127187484431583


In [16]:
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

In [15]:
XGB_model_final = XGBRegressor(max_depth = 15, n_estimators = 200, learning_rate = 0.05, subsample = 0.8, colsample_bytree = 1, random_state = 42)

In [19]:
pipeline = Pipeline([('XGB_model_final', XGBRegressor(max_depth=15, n_estimators=200, learning_rate=0.05, subsample=0.8, colsample_bytree=1, random_state=42))])

In [20]:
cv_results = cross_validate(pipeline, X, y_log, cv=10, scoring=['r2', 'neg_root_mean_squared_error'], return_train_score=True)

In [22]:
print("Mean R^2:", cv_results['test_r2'].mean())
print("Mean RMSE:", -cv_results['test_neg_root_mean_squared_error'].mean()) #MSE = 0.1142

Mean R^2:  0.7183553653369954
Mean RMSE:  0.3378757203231053


The MSE of the tuned random forest was 0.117 and the MSE of the tuned gradient boosted model was 0.114. The R^2 of the tuned random forest was 0.710 and the R^2 of the gradient boosted model was 0.718. Clearly, even the incorporation of a more robust tree model did not significantly change the error and amount of variance explained by the model. So, to make more accurate predictions, a different model, such as a neural network, must be used.